In [ ]:
allSamples_geno = open(snakemake.input["as_geno"], "r").readlines()

In [ ]:
split_index_dict = {n: [] for n in snakemake.params["splits"]}
num_splits = len(snakemake.params["splits"])
idx_array = [x for x in range(len(allSamples_geno))]

for n in range(num_splits):
    idx = [x for x in range(n, len(idx_array), num_splits)]
    split_index_dict[n] = [idx_array[i] for i in idx]

In [ ]:
out_prefix = "/research/projects/trifolium/glue/glue_ps/results/program_resources/baypass/split_files/"
for split, geno in split_index_dict.items():
    as_geno_out = out_prefix + f"allSamples/splits/allSamples_{split}.geno"
    with open(as_geno_out, "w") as fout:
        for g in geno:
            fout.write(allSamples_geno[g])

In [ ]:
site_order = open(snakemake.input["site_order"], "r").readlines()
for split, geno in split_index_dict.items():
    site_order_out = out_prefix + f"allSamples/site_order/site_order_{split}.txt"
    with open(site_order_out, "w") as fout:
        for g in geno:
            fout.write(site_order[g])

In [ ]:
city_in_prefix = "/research/projects/trifolium/glue/glue_ps/results/program_resources/baypass/"
for city in snakemake.params["cities"]:
    city_in_path = f"{city_in_prefix}/allSites/{city}/{city}.geno"
    city_geno = open(city_in_path, "r").readlines()
    for split, geno in split_index_dict.items():
        city_geno_out = out_prefix + f"byCity/{city}/{city}_{split}.geno"
        with open(city_geno_out, "w") as fout:
            for g in geno:
                fout.write(city_geno[g])